#  Code for 2D staggered Lagrangian method in Cartersian coordinates

## Control equations

### Momentum equation

$$\tag{1.1}\rho \frac{d\mathbf{v}}{dt} =  -\nabla p $$ 
where $\mathbf{v}$ is the vector of velocity, and $\rho$ is the density and $p$ is the pressure.  

### Energy equation

$$ \rho \frac{dE}{dt} = -\nabla \cdot (p\mathbf{v})$$ 

where $E$ is the specific total energy and it has a realtion with the specific internal energy 
$$E = e + \frac{1}{2}\mathbf{v}\cdot\mathbf{v}$$

By (1.1) and (1.2) we have 
$$\tag{1.3} \rho \frac{de}{dt} = - p\nabla\cdot \mathbf{v}$$

In  a Cartersian coordinates $x-y$, The equations (1.1) and (1.3) can be written as
$$\rho \frac{du}{dt} = -\frac{\partial p}{\partial x},\quad \rho \frac{dv}{dt} = -\frac{\partial p}{\partial y}$$
and
$$\rho \frac{de}{dt} = -p (\frac{\partial u}{\partial x} +\frac{\partial v}{\partial y})$$

## Compatible Staggered Discretization

<img src="Grid.png" width = "800" height = "400" div align=center /> 

<img src="cellpoint.png" width = "800" height = "400" div align=center />

### Quatities:

On point: ($x_p$,$y_p$), ($u_p$,$v_p$)

Zonal:  $V_z$, $\rho_z$, $m_z$, $e_z$, $p_z$

By the relation 
$$\frac{dm_z}{dt} =0$$
we have 
$$\rho_z(t) = m_z/V_z(t)$$

Also 
$$d(x_p)/dt = u_p, \quad d(y_p)/dt = v_p$$

### Point mass 

Point mass $m_p$ is used to solve point momentum $\mu_p = m_p \bf{u}_p$ and kinetic energy $K_p = m_p \frac{\bf{u}_p^2}{2}$

<img src="Ins.jpg" width = "200" height = "400" div align=center /> 

$$m_p = \sum_{z\in Z(p)} A_z^p \rho^p_z$$

$$\rho_z(t) = m_z/V_z(t)$$

In [2]:
function MassAndDens(mz::Array{Float64,1},x::Array{Float64,1},y::Array{Float64,1})
    
    Ic, = size(mz)
    Ip, = size(Tc)
    Asbz = zeros(Float64,4)
    ρ = zeros(Float64,Ic)
    mp = zeros(Float64,Ip)
    xz = zeros(Float64,4)
    yz = zeros(Float64,4)
    for i in 1:Ic
        xz[1:4] = x[Tp[i,1:4]]
        yz[1:4] = y[Tp[i,1:4]]
        Asbz = A_z(xz,yz)
        Az = sum(Asbz)
        ρ[i] = mz[i]/Az
        
        for j = 1:4
            ip = Tp[i,j]
            mp[ip] = mp[ip] + Asbz[j]*ρ[i]
        end
    end
   # @show mp
    return ρ,mp
    
   
end           
        

MassAndDens (generic function with 1 method)


$$ A_z^1 = \frac{5A_{41}+5A_{12}+A_{23}+A_{34}}{12}$$
$$ A_z^2 = \frac{A_{41}+5A_{12}+5A_{23}+A_{34}}{12}$$
$$ A_z^3 = \frac{A_{41}+A_{12}+5A_{23}+5A_{34}}{12}$$
$$ A_z^4 = \frac{5A_{41}+A_{12}+A_{23}+5A_{34}}{12}$$

In [3]:
function A_z(xz::Array{Float64,1},yz::Array{Float64,1}) 
    A =zeros(Float64,4)
    xc= sum(xz[i] for i in 1:4)/4
    yc= sum(yz[i] for i in 1:4)/4
    
    A41 = Area3(xz[4],xz[1],xc,yz[4],yz[1],yc)
    A12 = Area3(xz[1],xz[2],xc,yz[1],yz[2],yc)
    A23 = Area3(xz[2],xz[3],xc,yz[2],yz[3],yc)
    A34 = Area3(xz[3],xz[4],xc,yz[3],yz[4],yc)

    A[1] = (5A41+5A12+A23+A34)/12
    A[2] = (A41+5A12+5A23+A34)/12
    A[3] = (A41+A12+5A23+5A34)/12
    A[4] = (5A41+A12+A23+5A34)/12
    
    

    return A
end

A_z (generic function with 1 method)

<img src="Ins2.jpg" width = "400" height = "300" div align=center />

In [29]:
function F_z(x::Array{Float64,1},y::Array{Float64,1},p::Float64)
   
    Fx = zeros(Float64,4)
    Fy = zeros(Float64,4)
    
    L12 = √((x[2]-x[1])^2+(y[2]-y[1])^2)
    n12x = (-y[2]+y[1])/L12
    n12y = (x[2]-x[1])/L12
    
    L41 = √((x[1]-x[4])^2+(y[1]-y[4])^2)
    n41x = (-y[1]+y[4])/L41
    n41y = (x[1]-x[4])/L41
    
    L23 = √((x[3]-x[2])^2+(y[3]-y[2])^2)
    n23x = (-y[3]+y[2])/L23
    n23y = (x[3]-x[2])/L23
    
    L34 = √((x[4]-x[3])^2+(y[4]-y[3])^2)
    n34x = (-y[4]+y[3])/L34
    n34y = (x[4]-x[3])/L34
    
    Fx[1] = p*(L12*n12x+L41*n41x)/2
    Fy[1] = p*(L12*n12y+L41*n41y)/2
    
    Fx[2] = p*(L12*n12x+L23*n23x)/2
    Fy[2] = p*(L12*n12y+L23*n23y)/2
    
    Fx[3] = p*(L23*n23x+L34*n34x)/2
    Fy[3] = p*(L23*n23y+L34*n34y)/2
    
    Fx[4] = p*(L34*n34x+L41*n41x)/2
    Fy[4] = p*(L34*n34y+L41*n41y)/2
    
    return Fx,Fy
end           

F_z (generic function with 1 method)

对于单元 
$$ m_z \frac{de_z}{dt} = -\sum_{z\in Z(p)} \bf{f}_z^p \cdot \bf{u}_p $$  

对于节点
$$m_p \frac{d\bf{u}_p}{dt} = \sum_{p\in P(z)} \bf{f}_z^p$$

In [5]:
function Rhs(x::Array{Float64,1},y::Array{Float64,1},u::Array{Float64,1},v::Array{Float64,1},p::Array{Float64,1})  #Right hand e
    Ic, = size(Tp)
    Ip, = size(Tc)
    rhsu = zeros(Float64,Ip)
    rhsv = zeros(Float64,Ip) 
    rhse = zeros(Float64,Ic)
    xz = zeros(Float64,4)
    yz = zeros(Float64,4)
    
    for i in 1:Ic
        for j =1:4
            xz[j] = x[Tp[i,j]]
            yz[j] = y[Tp[i,j]]
        end
        Fx,Fy = F_z(xz,yz,p[i])
        
        ∑=0
        for j in 1:4
            ip = Tp[i,j]
            ∑ = ∑ +u[ip]*Fx[j] + v[ip]*Fy[j]
        end
        rhse[i] = ∑
        
        for j = 1:4
            ip = Tp[i,j]
            rhsu[ip] +=  -Fx[j]
            rhsv[ip] +=  -Fy[j]
        end     
    end
    return rhse, rhsu,rhsv
end

Rhs (generic function with 1 method)

In [7]:
function Euler1st(dt,u,v,x,y,p,mz,ρ,α)
    Ip, = size(u)
    Ic, = size(p)
    
    ρ,mp = MassAndDens(mz,x,y)
   
 #   @show mp
    
    e = pToe(p, ρ,problem.γ)
    
    p = artificial_viscosity!(p,ρ,x,y,u,v,α)
    
    rhse, rhsu,rhsv = Rhs(x,y,u, v, p)
    
    rhsu,rhsv = Bound(rhsu,rhsv,x,y,u, v, p)
    
    x  +=  dt*u
    y  +=  dt*v
    u  += dt*(rhsu ./ mp)
    v  += dt*(rhsv ./ mp)
    e  += dt*(rhse ./ mz)
    

    ρ,mp = MassAndDens(mz,x,y)
    
    p = eTop(e, ρ,problem.γ)
    return u,v,x,y,p,ρ
end

Euler1st (generic function with 1 method)

In [8]:
function Bound(rhsu,rhsv,x,y,u,v,p)
    Ip, = size(Tc)
    Ib, = size(Tb)
    xz = zeros(Float64,4)
    yz = zeros(Float64,4)
    for i in 1:Ib
        ip = Tb[i,1]
    #    @show ip
        for j = 1:4
            ic = Tc[ip,j]
            if ic != 0
                xz[1:4] = x[Tp[ic,1:4]]
                yz[1:4] = y[Tp[ic,1:4]]
                if Tb[i,2] ==1
                    Fx,Fy = F_zB(xz,yz,p[ic],j,1)
                    rhsu[ip] +=  Fx
                    rhsv[ip] += Fy
                   # mp[ip] *= 2.0
                end
                if Tb[i,3] ==1
                    Fx,Fy = F_zB(xz,yz,p[ic],j,2)
                    rhsu[ip] += Fx
                    rhsv[ip] += Fy
                   # mp[ip] *= 2.0
                end
            end
        end
    end
    return rhsu, rhsv
end  

Bound (generic function with 1 method)

In [33]:
x,y,u,v,mz,p,ρ,Az,α=InitTest2()
#Output1
u,v,x,y,p,ρ = TimeSolve(u, v, x, y, p, mz,ρ,α)
Output(x, y, u, v, p, ρ, mz)

0.0001056442818410635
0.00020023276674526434
0.00028583093983474263
0.0003640399038547588
0.00043609971275410597
0.0005029790897279289
0.0005654444314048043
0.0006241104928227396
0.0006794771136073401
0.0007319558653758672
0.0007818895530391861
0.0008295666596201969
0.0008752321915600479
0.0009190959354781524
0.0009613388304910707
0.0010021179505626352
0.0010415704478948919
0.001079816709538555
0.0011169629106783923
0.0011531030997402219
0.0011883209161115485
0.0012226910165446378
0.0012562802683067605
0.0012891487538782235
0.0013213506221163981
0.0013529348133613105
0.0013839456802954952
0.0014144235220200675
0.0014444050454356626
0.0014739237653788083
0.0015030103528845139
0.0015316929392937342
0.0015599973826025994
0.0015879475013856914
0.0016155652807627603
0.0016428710541745994
0.0016698836641566937
0.001696620604823307
0.0017230981483800428
0.001749331457654015
0.0017753346863552371
0.0018011210685508973
0.0018267029986379472
0.001852092102932672
0.00187729930385347
0.00190233487

0.010349822909629604
0.010376484246261057
0.010403147907818327
0.010429813819979194
0.010456481873714814
0.01048315192791453
0.010509823812001719
0.01053649728925671
0.010563167998421172
0.010589836009340981
0.010616501390622163
0.010643164209551356
0.010669824532053633
0.010696482422682842
0.010723137944639537
0.01074978996834153
0.010776438511821976
0.010803083639052606
0.010829725413012836
0.010856363895787776
0.01088299914865315
0.010909631232150233
0.010936260206147162
0.010962886129879739
0.010989509061978481
0.011016129060486777
0.011042746182873648
0.011069360486043614
0.011095972026345413
0.011122580859580783
0.011149187041014133
0.01117579062538358
0.011202391666913667
0.011228990219329858
0.011255586335874788
0.011282180069326174
0.011308771472016161
0.011335360144230396
0.01136194537699856
0.011388527235635475
0.011415105785168609
0.011441681090320155
0.011468253215490724
0.011494822224744464
0.011521388181795423
0.011547951149994918
0.011574511192319694
0.01160106837136058

0.020798768085256013
0.020824874853566964
0.020850980979409954
0.020877086465775557
0.02090319131560299
0.020929295531781685
0.020955399117152815
0.020981502002215137
0.02100760417244766
0.02103370563253048
0.021059806387045965
0.02108590644048153
0.021112005797232332
0.021138104461603895
0.021164202437814687
0.021190299729998602
0.0212163963422074
0.021242492278413072
0.021268587542510167
0.021294682138318026
0.021320776069582985
0.02134686933998052
0.021372961953117317
0.021399053912533314
0.021425145221703686
0.02145123588404076
0.021477325902895906
0.021503415281561366
0.021529504023272052
0.021555592131207273
0.021581679608492433
0.021607766458200693
0.02163385268335458
0.021659938286927565
0.021686023271845576
0.021712107640988517
0.0217381913971917
0.021764274543247267
0.021790357081905583
0.021816439015876567
0.02184252034783101
0.021868601080401857
0.02189468121618544
0.021920760757742706
0.021946839707600387
0.02197291806825215
0.021998995842159728
0.022025073031753998
0.0220

0.031175905858136357
0.03120184425847141
0.031227782423089982
0.03125372035306519
0.03127965804946449
0.03130559551334974
0.031331532745777234
0.03135746974779769
0.031383406520456354
0.03140934306479297
0.031435279381841846
0.03146121547263189
0.031487151338186636
0.031513086979524256
0.03153902239765763
0.031564957593594345
0.03159089256735632
0.03161682731973491
0.03164276185168254
0.03166869616414603
0.03169463025806665
0.031720564134188216
0.03174649779331763
0.03177243123636845
0.031798364464248614
0.031824297477860516
0.03185023027810104
0.031876162865861604
0.031902095242028194
0.03192802740748142
0.03195395936309656
0.031979891109743586
0.03200582264828725
0.032031753979587095
0.032057685104497524
0.03208361601866976
0.032109546712513024
0.03213547718700483
0.03216140744311781
0.032187337481819775
0.03221326730407375
0.03223919691083803
0.03226512630306622
0.03229105548170732
0.03231698444770571
0.03234291320200126
0.032368841745529335
0.032394770079220876
0.03242069820400243


0.04166468145363186
0.04169052463424182
0.04171636714735538
0.04174220905109536
0.04176805040442447
0.04179389126686736
0.04181973169823879
0.04184557175837945
0.04187141150690102
0.041897251002941956
0.04192309030493519
0.04194892947038917
0.041974768555683155
0.042000607615877976
0.042026446393188095
0.04205228451624406
0.042078122044919536
0.042103959039847955
0.04212979556213989
0.04215563167310563
0.04218146743398482
0.04220730290568509
0.04223313814853182
0.04225897322203055
0.04228480818464359
0.04231064309358233
0.04233647800461635
0.042362312719928956
0.042388147040706765
0.042413980802424006
0.04243981312279089
0.042465644066020145
0.04249147370055793
0.042517302098690334
0.04254312933613673
0.04256895549162969
0.042594780646482644
0.04262060488415017
0.04264642828978266
0.04267225094977821
0.04269807295133944
0.04272389438203831
0.04274971532939175
0.042775535880451
0.04280135612140703
0.04282717613721447
0.042852996011236134
0.04287881582491007
0.04290463565744085
0.0429304

...

0.073876605580254
0.0738767880837212
0.07387697295185536
0.07387716021960689
0.07387734992246267
0.07387754209645604
0.07387773677817686
0.07387793400478185
0.07387813381400513
0.07387833624416897
0.07387854133419477
0.07387874912361433
0.0738789596525812
0.07387917296188248
0.07387938909295072
0.07387960808787615
0.07387982998941914
0.07388005484102293
0.07388028268682667
0.07388051357167869
0.0738807475411501
0.07388098464154867
0.07388122491993307
0.0738814684241273
0.07388171520273555
0.07388196530515737
0.07388221878160313
0.07388247568310989
0.07388273606155756
0.07388299996968549
0.07388326746110938
0.0738835385903386
0.0738838134127939
0.07388409198482554
0.07388437436373177
0.07388466060777782
0.07388495077621529
0.07388524492930197
0.07388554312832214
0.07388584543560733
0.0738861519145576
0.07388646262966322
0.07388677764652704
0.07388709703188719
0.0738874208536404
0.07388774918086596
0.07388808208385007
0.07388841963411091
0.07388876190442425
0.07388910896884975
0.07388946

0.07417698218115443
0.07417768230124631
0.07417837021869772
0.07417904708632714
0.07417971402684317
0.07418037213507579
0.07418102248010103
0.07418166610726594
0.07418230404011983
0.07418293728225743
0.07418356681907891
0.0741841936194713
0.07418481863741523
0.07418544281352035
0.07418606707649242
0.07418669234453427
0.07418731952534435
0.07418794951863462
0.0741885832174143
0.07418922150915652
0.07418986527690487
0.07419051540031947
0.07419117275666251
0.07419183822172222
0.07419251267067421
0.07419319697887868
0.07419389202261141
0.07419459867972644
0.07419531783024742
0.07419605035688479
0.07419679714547499
0.07419755908533786
0.07419833706954766
0.07419913199511277
0.07419994476305855
0.07420077627840735
0.07420160644740402
0.074202429354381
0.07420324506102603
0.07420405362864808
0.07420485511817396
0.07420564959014506
0.07420643710471454
0.07420721772164458
0.07420799150030394
0.07420875849966573
0.07420951877830544
0.07421027239439912
0.07421101940572178
0.07421175986964601
0.07

0.07429553719687372
0.07429559350255047
0.07429564973643431
0.0742957059023769
0.07429576200422379
0.07429581804581471
0.07429587403098381
0.07429592996355994
0.0742959858473669
0.07429604168622371
0.07429609748394489
0.07429615324434069
0.07429620897121737
0.07429626466837747
0.07429632033962003
0.07429637598874089
0.07429643161953296
0.07429648723578643
0.07429654284128903
0.07429659843982636
0.07429665403518207
0.07429670963113816
0.0742967652314752
0.07429682083997265
0.07429687646040901
0.07429693209656221
0.07429698775220976
0.07429704343112904
0.07429709913709757
0.07429715487389324
0.07429721064529457
0.07429726645508099
0.07429732230703305
0.07429737820493271
0.07429743415256357
0.07429749015371115
0.0742975462121631
0.0742976023317095
0.07429765851614309
0.07429771476925953
0.07429777109485763
0.07429782749673965
0.07429788397871147
0.07429794054458295
0.07429799719816811
0.0742980539432854
0.07429811078375795
0.07429816772341383
0.07429822476608629
0.07429828191561402
0.0742

0.07437628992352593
0.07437688876557405
0.07437749148831442
0.0743780981060336
0.0743787086328982
0.074379323082952
0.07437994147011313
0.07438056380817112
0.0743811901107838
0.0743818203914744
0.07438245466362868
0.07438309294049204
0.07438373523516663
0.07438438156060845
0.07438503192962446
0.07438568635486974
0.07438634484884454
0.07438700742447642
0.07438767409622268
0.07438834487827879
0.07438901978456887
0.07438969882873613
0.07439038202413302
0.07439106938381132
0.07439176092051193
0.07439245664665455
0.0743931565743271
0.07439386071527512
0.07439456908089077
0.07439528168220183
0.0743959985298604
0.07439671963413147
0.07439744500488134
0.07439817465156578
0.07439890858321811
0.07439964680843696
0.07440038933537403
0.07440113617172157
0.07440188732469973
0.07440264280104365
0.0744034026069906
0.0744041667482667
0.07440493523007372
0.07440570805707551
0.0744064852333845
0.0744072667625479
0.0744080526475338
0.07440884289071723
0.07440963749386593
0.07441043645812623
0.07441123978

DomainError: DomainError with -0.003764811944891439:
sqrt will only return a complex result if called with a complex argument. Try sqrt(Complex(x)).

In [10]:
Output(x, y, u, v, p, ρ, mz)

UndefVarError: UndefVarError: Output not defined

### Boundary force

If one edge of a cell is a free boundary, then $f$ on the points is zero. As we have add it in RHs(), we need to remove it by resolve it again. Different from RHs(), at every point of a cell the force is composited by two parts on different edges. This is distinguished by $k$ when $k = 1$ the boundary is on y direction, forces on 12 and 34 boundary must be subtracted. When $ k =2$ the boundary is on x direction, subtract forces on 41, 23. 

In [11]:
function F_zB(x::Array{Float64,1},y::Array{Float64,1},p::Float64,j::Int,k::Int)
    Fx = 0.0
    Fy = 0.0      
    if k==1 # y direction 
        L12 = √((x[2]-x[1])^2+(y[2]-y[1])^2)
        n12x = (-y[2]+y[1])/L12
        n12y = (x[2]-x[1])/L12
        L34 = √((x[4]-x[3])^2+(y[4]-y[3])^2)
        n34x = (-y[4]+y[3])/L34
        n34y = (x[4]-x[3])/L34
        if j == 1  #Point 1
            Fx = p*(L12*n12x)/2
            Fy = p*(L12*n12y)/2
        elseif j == 2 
            Fx = p*(L12*n12x)/2
            Fy = p*(L12*n12y)/2
        elseif j == 3
            Fx = p*(L34*n34x)/2
            Fy = p*(L34*n34y)/2
        else 
            Fx = p*(L34*n34x)/2
            Fy = p*(L34*n34y)/2
        end
    elseif k == 2
        L41 = √((x[1]-x[4])^2+(y[1]-y[4])^2)
        n41x = (-y[1]+y[4])/L41
        n41y = (x[1]-x[4])/L41

        L23 = √((x[3]-x[2])^2+(y[3]-y[2])^2)
        n23x = (-y[3]+y[2])/L23
        n23y = (x[3]-x[2])/L23
        if j == 1
            Fx = p*(L41*n41x)/2
            Fy = p*(L41*n41y)/2
        elseif j == 2 
            Fx = p*(L23*n23x)/2
            Fy = p*(L23*n23y)/2
        elseif j == 3
            Fx = p*(L23*n23x)/2
            Fy = p*(L23*n23y)/2
        else 
            Fx = p*(L41*n41x)/2
            Fy = p*(L41*n41y)/2
        end
    end
    return Fx,Fy
    end           

F_zB (generic function with 1 method)

### EOS 
$$ e = \frac{p}{(\gamma-1)\rho}$$

In [12]:
function pToe(p,ρ,γ)
    return p ./ ρ/(γ-1)
end
function eTop(e,ρ,γ)
    return (γ-1)*ρ .* e
end

eTop (generic function with 1 method)

In [13]:
function CFL(SF,x::Array{Float64,1},y::Array{Float64,1},
                u::Array{Float64,1},v::Array{Float64},
                p::Array{Float64,1},ρ::Array{Float64,1})
    Ic, = size(ρ)
    cflmin = 1.e9
      cfl = 0 
    γ = problem.γ
    for i in 1:Ic
        cfl =0
        u2max = 0
        u2=0.0
        for j = 1:4
            ip = Tp[i,j]
            u2 = u[ip]^2+v[ip]^2
            if u2 > u2max ; u2max = u2; end
        end
        
        dlmin = (x[Tp[i,4]] - x[Tp[i,1]])^2+(y[Tp[i,4]]-y[Tp[i,1]])^2
        for j=1:3
            dl= (x[Tp[i,j+1]] - x[Tp[i,j]])^2+(y[Tp[i,j+1]]-y[Tp[i,j]])^2
            if dl < dlmin; dlmin=dl; end
        end
        c = √(γ*p[i]/ρ[i])
        cfl = √(dlmin)/(√(u2max)+c)
        if cfl < cflmin; cflmin = cfl; end
    end
 #   @show cflmin
    dt = cflmin*SF
    return dt
end 
        

CFL (generic function with 1 method)

In [14]:
function TimeSolve(u,v,x,y,p,mz,ρ,α)
    tt= problem.tt
    sf =problem.sf
    t= 0.0
    while t<tt
#for i in 1:50
        dt=CFL(sf,x,y,u,v,p,ρ)
        if t+dt>tt
           dt = tt-t
        end
        
        u,v,x,y,p,ρ = Euler1st(dt,u,v,x,y,p,mz,ρ,α)
        t += dt
        
        println(t)
    end
#
    return u,v,x,y,p,ρ
end

TimeSolve (generic function with 1 method)

## Artificial Viscosity 

Wilkins（1980） use the strain rate along the direction of cell acceleration to construct the artificial viscosity 
$$\frac{ds}{dt} = \frac{\partial u}{\partial x} \text{cos}^2 \alpha + \frac{\partial v}{\partial y} \text{sin}^2\alpha +(\frac{\partial u}{\partial y} + \frac{\partial v}{\partial x})\text{sin} \alpha \text{cos} \alpha$$


Then the artificial viscosity can be written as
$$ q_w = \left\{ \begin{align}
l_0^2 \rho \left( \frac{ds}{dt} \right)^2 +  l_L\rho c \left|\frac{ds}{dt}\right|, \quad \text{if}  \quad \frac{ds}{dt}< 0,\\
0, \quad \text{if} \quad \frac{ds}{dt}\ge 0. \\
\end{align}
\right.
$$
where $l_0 = a_0L$, $l_L = a_L L$, $a_0 \approx 2$, $a_L \approx 1$ and $c$ is the sonic speed.

$L$ is the reference length, and constructed as
$$L = \frac{2A}{d_1 + d_2 +d_3 +d_4} $$


<img src="L1.png" width = "400" height = "300" div align=center />

In [15]:
function artificial_viscosity_c!(p,ρ,x,y,u,v,α)
    γ = problem.γ
    xc = sum(x[i] for i in 1:4)/4
    yc = sum(y[i] for i in 1:4)/4
    
    ∂u_∂x,∂u_∂y,∂v_∂x,∂v_∂y = ∂uv(x,y,u,v)
    
    ds_dt = ∂u_∂x*cosd(α)^2 +∂v_∂y*sind(α)^2 + (∂u_∂y+∂v_∂x)*sind(α)*cosd(α)
    
    L = l_viscosity(x,y,α)
    a₀ = 2; aₗ = 1
    c = √(γ*p/ρ)
    l₀ = a₀*L
    lₗ = aₗ*L
    
    L = abs(x[3] - x[1])
    
   # if ds_dt > 0
    if ∂u_∂x > 0
        q = 0
    else
        #q = l₀^2*ρ*(ds_dt)^2 +lₗ*ρ*c*abs(ds_dt)
        q = -1.5ρ*L*c*∂u_∂x +0.06L^2*ρ*(∂u_∂x)^2 
    end
    return p+q
end

artificial_viscosity_c! (generic function with 1 method)

$$b_2 \rho Lc \frac{\partial u}{\partial x} - b_{12}\rho L^2 \left(\frac{\partial u}{\partial x}\right)^2$$

In [16]:
function artificial_viscosity!(p,ρ,x,y,u,v,α)
    I, = size(Tp)
    
    xc = zeros(Float64,4)
    yc = zeros(Float64,4)
    uc = zeros(Float64,4)
    vc = zeros(Float64,4)
    for i in 1:I
        for j in 1:4
            ip = Tp[i,j]
            xc[j] = x[ip]
            yc[j] = y[ip]
            uc[j] = u[ip]
            vc[j] = v[ip]
        end
        p[i] = artificial_viscosity_c!(p[i],ρ[i],xc,yc,uc,vc,α[i])
    end
    
        return p
        
    end       

artificial_viscosity! (generic function with 1 method)

In [17]:
function l_viscosity(x,y,α)
    xc = sum(x)/4; yc = sum(y)/4
    d = zeros(Float64,4)
    for i in 1:4
    d[i] = distance(x[i],y[i],xc,yc,α)
    end
    A = A_z(x, y)
    L = 2sum(A)/sum(d)
    return L
end

l_viscosity (generic function with 1 method)

#### distance from  Point to line 
Line: $$ (xc -cos \alpha )(x-xc) = (yc-sin \alpha)(y-yc)$$
point: $x,y$

In [18]:
function distance(x,y,xc,yc,α)
    d = abs((xc - cosd(α)) * (x .-xc)- (yc-sind(α))*(y .- yc))/√((xc-cosd(α))^2+(yc-sind(α))^2)
    return d
end

distance (generic function with 1 method)

In [19]:
struct prb
    tt::Float64
    sf::Float64
    γ ::Float64
end

In [20]:
function Area3(x1,x2,x3,y1,y2,y3)
    return abs((x1*y2+y1*x3+x2*y3-x1*y3-y1*x2-y2*x3)/2)
end

Area3 (generic function with 1 method)

###  $\frac{\partial (u,v)}{\partial (x,y)}$

we use the diffences between 1 and 3 points and 2 and 4 points to solve the partial differences approximately, as
$$ \frac{\partial u}{\partial x}  \approx \frac{1}{2}\frac{u_3-u_1}{x_3-x_1}+\frac{1}{2}\frac{u_4-u_2}{x_4-x_2}$$ 
Similar process to $\frac{\partial u}{\partial y}$,$\frac{\partial v}{\partial x}$ and $\frac{\partial v}{\partial y}$

In [21]:
function ∂uv(x,y,u,v)
    ∂u_∂x = (u[3]-u[1])/(x[3]-x[1]) #+ 0.5(u[2]-u[4])/(x[2]-x[4])
    ∂v_∂x = 0.5(v[3]-v[1])/(x[3]-x[1]) + 0.5(v[2]-v[4])/(x[4]-x[2])
    ∂u_∂y = 0.5(u[3]-u[1])/(y[3]-y[1]) + 0.5(u[4]-u[2])/(y[4]-y[2])
    ∂v_∂y = 0.5(v[3]-v[1])/(y[3]-y[1]) + 0.5(v[4]-v[2])/(y[4]-y[2])
    return ∂u_∂x,∂u_∂y,∂v_∂x,∂v_∂y
end

∂uv (generic function with 1 method)

### noh 算例 

In [22]:
@inline function InitTest1()

    global problem = prb(0.1,0.3,1.4)
     
    I = 200
    J = 20
    dx = 1.0/I
    dy = 0.1/J
    
    
    Ip = (I+1)*(J+1) # Number of  points
    Ic = I*J  # number of cells
    
    global Tp = zeros(Int,Ic,4) #格点
    global Tc = zeros(Int,Ip,4) #点格
    
    #@show Tc, Ip
    global Tb = zeros(Int, I*2+J*2,3) #边界 
    
    IBL = J+1
    IBR = J+1
    IBU = I+1
    IBD = I+1
   # 点格表，格点表初始化 
    for i in 1:I 
        for j in 1:J
            ic = i+(j-1)*I
            Tp[ic,1] = i + (j-1)*(I+1)
            Tp[ic,2] = i+1 +(j-1)*(I+1)
            Tp[ic,3]=i+1+j*(I+1)
            Tp[ic,4]=i+j*(I+1) 
        end
    end
    
    for i = 1:I+1
        for j = 1:J+1
            ip = i+(j-1)*(I+1)
            Tc[ip,1] = i +(j-1)*I #(i,j)
            Tc[ip,2] = i-1+(j-1)*I #(i-1,j)
            Tc[ip,3] = i-1+(j-2)*I #(i-1,j-1)
            Tc[ip,4] = i+(j-2)*I   #(i,j-1)
            end
        end
    

# Bound 
  ib = 0
    for i = 1:I+1
        for j = 1:J+1
            if i == 1 || i== I+1 ||  j == 1 || j == J+1
                ib += 1
                ip =i+(j-1)*(I+1) 
                Tb[ib,1] = ip
                if i==1 
                   Tb[ib,3] = 1
                    Tc[ip,2] =0
                    Tc[ip,3] =0
                end   
                 if  i==I+1 
                    Tb[ib,3] = 1 #bound type 1:free 2: wall ..  Tb[:,2] y direction
                    Tc[ip,1] =0
                    Tc[ip,4] =0
                end
            
                if j==1 
                    Tb[ib,2] = 1
                    Tc[ip,3] = 0
                    Tc[ip,4] = 0
                end
                if j==J+1
                    Tb[ib,2] = 1
                    Tc[ip,1] = 0
                    Tc[ip,2] = 0
                end
            end
        end
    end

    #流场初始化 Init of the flow
    
    x = zeros(Float64,Ip)
    y = zeros(Float64,Ip)
    u = zeros(Float64,Ip)
    v = zeros(Float64,Ip)
    
    ρ = zeros(Float64,Ic)
    mz = zeros(Float64,Ic)
    p = zeros(Float64,Ic)
    Az= zeros(Float64,Ic)
    α = zeros(Float64,Ic)
    for i in 1:I+1
        for j = 1:J+1
            ip =i+(j-1)*(I+1)
            x[ip] = i*dx
            y[ip] = j*dy
            
            u[ip] = 1.0
            v[ip] = 0.0
        end
    end
    
    xz = zeros(Float64,4)
    yz = zeros(Float64,4) 
    for i in 1:I
        for j in 1:J
            ic = i+(j-1)*I
            xz[1:4] = x[Tp[ic,1:4]]
            yz[1:4] = y[Tp[ic,1:4]]
            A = A_z(xz,yz)
            Az[ic] = sum(A) #[1]+A[2]+A[3]+A[4]
          #  if i<=100
                mz[ic] = (2+sin(2π*dx*i)) * Az[ic]
                p[ic] = 1.0
                ρ[ic] = sin(2π*dx*i)+2
#             else
#                 mz[ic] = 0.1 * Az[ic]
#                 p[ic] = 1.0
#                 ρ[ic] = 0.1
#             end
        end
    end
    
    return x,y,u,v,mz,p,ρ,Az,α
end

InitTest1 (generic function with 1 method)

In [32]:
@inline function InitTest2()

    global problem = prb(0.1,0.1,1.4)
     
    I = 800
    J = 20
    dx = 1.0/I
    dy = 0.1/J
    
    
    Ip = (I+1)*(J+1) # Number of  points
    Ic = I*J  # number of cells
    
    global Tp = zeros(Int,Ic,4) #格点
    global Tc = zeros(Int,Ip,4) #点格
    
    #@show Tc, Ip
    global Tb = zeros(Int, I*2+J*2,3) #边界 
    
    IBL = J+1
    IBR = J+1
    IBU = I+1
    IBD = I+1
   # 点格表，格点表初始化 
    for i in 1:I 
        for j in 1:J
            ic = i+(j-1)*I
            Tp[ic,1] = i + (j-1)*(I+1)
            Tp[ic,2] = i+1 +(j-1)*(I+1)
            Tp[ic,3]=i+1+j*(I+1)
            Tp[ic,4]=i+j*(I+1) 
        end
    end
    
    for i = 1:I+1
        for j = 1:J+1
            ip = i+(j-1)*(I+1)
            Tc[ip,1] = i +(j-1)*I #(i,j)
            Tc[ip,2] = i-1+(j-1)*I #(i-1,j)
            Tc[ip,3] = i-1+(j-2)*I #(i-1,j-1)
            Tc[ip,4] = i+(j-2)*I   #(i,j-1)
            end
        end
    

# Bound 
  ib = 0
    for i = 1:I+1
        for j = 1:J+1
            if i == 1 || i== I+1 ||  j == 1 || j == J+1
                ib += 1
                ip =i+(j-1)*(I+1) 
                Tb[ib,1] = ip
                if i==1 
                   Tb[ib,3] = 1
                    Tc[ip,2] =0
                    Tc[ip,3] =0
                end   
                 if  i==I+1 
                    Tb[ib,3] = 1 #bound type 1:free 2: wall ..  Tb[:,2] y direction
                    Tc[ip,1] =0
                    Tc[ip,4] =0
                end
            
                if j==1 
                    Tb[ib,2] = 1
                    Tc[ip,3] = 0
                    Tc[ip,4] = 0
                end
                if j==J+1
                    Tb[ib,2] = 1
                    Tc[ip,1] = 0
                    Tc[ip,2] = 0
                end
            end
        end
    end

    #流场初始化 Init of the flow
    
    x = zeros(Float64,Ip)
    y = zeros(Float64,Ip)
    u = zeros(Float64,Ip)
    v = zeros(Float64,Ip)
    
    ρ = zeros(Float64,Ic)
    mz = zeros(Float64,Ic)
    p = zeros(Float64,Ic)
    Az= zeros(Float64,Ic)
    α = zeros(Float64,Ic)
    for i in 1:I+1
        for j = 1:J+1
            ip =i+(j-1)*(I+1)
            x[ip] = i*dx
            y[ip] = j*dy
            if i >=100
                u[ip] = 0.
            else
                u[ip] = 0
            end
            v[ip] = 0.0
        end
    end
    
    xz = zeros(Float64,4)
    yz = zeros(Float64,4) 
    for i in 1:I
        for j in 1:J
            ic = i+(j-1)*I
            xz[1:4] = x[Tp[ic,1:4]]
            yz[1:4] = y[Tp[ic,1:4]]
            A = A_z(xz,yz)
            Az[ic] = sum(A) #[1]+A[2]+A[3]+A[4]
            if i>=100
                mz[ic] = 0.1 * Az[ic]
                p[ic] = 0.1
                ρ[ic] = 0.125
            else
                mz[ic] = 1.0 * Az[ic]
                p[ic] = 1.0
                ρ[ic] = 1.0
            end
        end
    end
    
    return x,y,u,v,mz,p,ρ,Az,α
end

InitTest2 (generic function with 1 method)

In [159]:
using DelimitedFiles
function Output(x,y,u,v,p,ρ,mz)

    Ip, = size(Tc)
    Ic, = size(Tp)
    io = open("data.dat", "w+")
    A = zeros(Float64,Ip,6)
   # for i in 1:Ip
    
    text = " TITLE = \"Dataset\"
VARIABLES = \"x\" \"y\" \"u\" \"v\" \"rho\" \"p\"  ZONE T=\"Zone 1\" 
I=201,J=21,K=1,ZONETYPE=Ordered 
DATAPACKING=POINT \n "
  #  write(1,*)
    write(io,text)

    for i in 1:Ip
        nc = 0
        ρz = 0.0
        pz = 0.0
        for j = 1:4
            ic = Tc[i,j] 
            if ic != 0
                ρz += ρ[ic]
                pz += p[ic]
                nc +=  1
            end
        end
        ρz = ρz/nc
        pz = pz/nc
        A[i,1] = x[i]
        A[i,2] = y[i]
        A[i,3] = u[i]
        A[i,4] = v[i]
        A[i,5] = ρz
        A[i,6] = pz
        
    end
    writedlm(io, A, "  ")
    close(io)
end

Output (generic function with 1 method)

In [27]:
using DelimitedFiles
function Output1(x,y,rhs)

    Ip, = size(Tc)
    Ic, = size(Tp)
    io = open("data1.dat", "w+")
    A = zeros(Float64,Ip,3)
   # for i in 1:Ip
    
    text = " TITLE = \"Dataset\"
VARIABLES = \"x\" \"y\" \"p\"  ZONE T=\"Zone 1\" 
I=400,J=40,K=1,ZONETYPE=Ordered 
DATAPACKING=POINT \n "
  #  write(1,*)
    write(io,text)

    for i in 1:Ic
        nc = 0
        ρz = 0.0
        pz = 0.0
        ip  = Tp[i,1]
        A[i,1] = x[ip]
        A[i,2] = y[ip]
        A[i,3] = rhs[i]
        
    end
    writedlm(io, A, "  ")
    close(io)
end

Output1 (generic function with 1 method)

In [24]:
using DelimitedFiles
function Output2(x,y,rhs)

    Ip, = size(Tc)
    Ic, = size(Tp)
    io = open("data1.dat", "w+")
    A = zeros(Float64,Ip,3)
   # for i in 1:Ip
    
    text = " TITLE = \"Dataset\"
VARIABLES = \"x\" \"y\" \"p\"  ZONE T=\"Zone 1\" 
I=401,J=41,K=1,ZONETYPE=Ordered 
DATAPACKING=POINT \n "
  #  write(1,*)
    write(io,text)

    for i in 1:Ip
        nc = 0
        ρz = 0.0
        pz = 0.0
       # ip  = Tp[i,1]
        A[i,1] = x[i]
        A[i,2] = y[i]
        A[i,3] = rhs[i]
        
    end
    writedlm(io, A, "  ")
    close(io)
end

Output2 (generic function with 1 method)

### 对流涡

### chenjuan shuqiwang 

In [99]:
using Plots   

In [100]:
scatter3d(x, y,ρ)

DimensionMismatch: DimensionMismatch("dimensions must match")

In [101]:
- 0.9/0.55*400

-654.5454545454545